1. Unpack both parts of the Yahoo dataset
2. Join them together (execute in command line / terminal): cat FUllOct2007.xml.part1 FullOct2007.xml.part2 > full.xml
3. Install xml_split: sudo apt-get install xml-twig-tools
4. Split the full file into approx. 1Gb big files: xml_split -s 1Gb full.xml

In [1]:
import pandas as pd
import numpy as np
import os
import re
from bs4 import BeautifulSoup
from html import unescape
from tqdm import tqdm
from dateutil.parser import parse
import xml.etree.ElementTree as ET

def clean(s):
    regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    s = re.sub(regex, "[REF]", s)
    s = unescape(BeautifulSoup(s,'lxml').get_text())
    if "<br />" in s:
        s=re.sub(r'(\n|<br />)', " ", s)
    else:
        s=re.sub(r'\n', "  ", s)
    s = re.sub(r"\'", "'", s, flags=re.MULTILINE)
    return s

In [2]:
tsv_file = os.path.join(
    os.getcwd(),
    "downloads/task2-stance-dataset/task2-stance-dataset/touche22-task2-stance-dataset.tsv",
)
df = pd.read_csv(tsv_file, sep="\t")
i = df[df["ds"] == "yahoo"].id.to_list()


In [3]:
df.head()

,ds,id,question,answer,answer_stance,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
0,softwareengineering.stackexchange,10373,What's the better way to charge for a cloud pl...,"Like all good questions, the answer depends. I...",2,simple but more expensive,2,simple but more expensive,"[[231, 252], [389, 393], [605, 619], [753, 757]]",complicated but cheaper,[]
1,softwareengineering.stackexchange,18838,Haskell AND Lisp vs. Haskell OR Lisp,"I suggest learning both, Haskell first, then C...",2,Haskell AND Lisp,2,Haskell AND Lisp,"[[25, 56]]",Haskell OR Lisp,[]
2,softwareengineering.stackexchange,19392,When is it better to offload work to the RDBMS...,You want to do all set-based operations in the...,3,do it in code,2,offload work to the RDBMS,"[[40, 55], [232, 248]]",do it in code,"[[165, 175], [578, 598]]"
3,softwareengineering.stackexchange,20653,Is it better to specialize in a single field I...,"Specialise if you enjoy it As you are aware, ...",3,expand into other fields to broaden my horizons,2,to specialize in a single field I like,"[[53, 63], [404, 410], [512, 519]]",expand into other fields to broaden my horizons,"[[892, 933]]"
4,softwareengineering.stackexchange,21186,Microsoft SDE Interview vs Microsoft SDET Inte...,"Unfortunately, those are both myths. SDEs and ...",1,Neutral,2,Microsoft SDE Interview,"[[37, 41], [283, 295], [950, 962]]",Microsoft SDET Interview,"[[46, 51], [341, 354], [832, 836], [938, 945]]"


In [ ]:
%%time

idx=""
question=[]
ids=[]
answers = []
ans_niklas=[]
ans_found=[]
uris=[]
found=""
for file in tqdm(os.listdir("full")): #path to the directory with the split dataset (e.g., the folder is called full)
    print(file)
    if '-' in file:
        tree = ET.parse("full/"+file)
        for r in tree.iter(tag='vespaadd'):
            r = r[0]
            uri=r.findtext('uri')
            if int(uri) in i:
                for idx, row in df.iterrows():
                    if int(uri)==row.id:
                        df.at[idx,'question'] = clean(r.findtext('subject'))
                        df.at[idx,'answer'] = clean(r.findtext('bestanswer'))

In [ ]:
df.head() # result df